In [ ]:
!pip install DeepFace

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 3.6 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=e95ac454d8e8cd52be7df37d3ce66f758d8f503c32c4a6db3391c6c4590ece50
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire


In [ ]:
import os
import cv2
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from deepface import DeepFace
from tensorflow import keras
from keras._tf_keras.keras.preprocessing.image import ImageDataGenerator
import joblib  # To save the model
import pandas as pd


In [ ]:
from datetime import datetime

# Path to your dataset
dataset_path = "/content/drive/MyDrive/Facial Dataset"

# Function to load and detect faces from the dataset (No augmentation of embeddings)
def load_and_detect_faces(folder_path, augment=False):
    face_images = []
    labels = []
    # Set up data augmentation if augment is True
    if augment:
        datagen = ImageDataGenerator(
            rotation_range=30,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode="nearest",
        )

    for label in os.listdir(folder_path):
        label_path = os.path.join(folder_path, label)
        if os.path.isdir(label_path):
            for file_name in os.listdir(label_path):
                file_path = os.path.join(label_path, file_name)
                # Read the image
                img = cv2.imread(file_path)
                if img is not None:
                    if augment:
                        # Augment the image
                        img = cv2.resize(img, (160, 160)) / 255.0  # Normalize the image
                        augmented_images = datagen.flow(
                            np.expand_dims(img, axis=0), batch_size=1
                        )
                        img = augmented_images.__next__()[0]

                    # Extract face embeddings using DeepFace
                    try:
                        # Using DeepFace to detect and get face embedding
                        result = DeepFace.represent(
                            img_path=file_path,
                            model_name="VGG-Face",
                            enforce_detection=False,
                        )
                        face_embedding = result[0]["embedding"]
                    except Exception as e:
                        print(f"Error processing {file_path}: {e}")
                        continue

                    face_images.append(face_embedding)
                    labels.append(label)

    return np.array(face_images), np.array(labels)


# Function to train Random Forest classifier
def train_rf_classifier(face_embeddings, labels):
    # Flatten the embeddings
    face_embeddings = face_embeddings.reshape(face_embeddings.shape[0], -1)

    # Encode labels
    le = LabelEncoder()
    labels = le.fit_transform(labels)

    # Train Random Forest classifier
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(face_embeddings, labels)

    # Save the model and label encoder
    joblib.dump(rf, "rf_classifier.pkl")  # Save Random Forest model
    joblib.dump(le, "label_encoder.pkl")  # Save Label Encoder

    return rf, le


# Function to mark attendance in a CSV file
def mark_attendance(name):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    attendance_data = pd.DataFrame([[name, timestamp]], columns=["Name", "Timestamp"])
    attendance_file = "attendance.csv"

    # Append to CSV or create it if it doesn't exist
    if not os.path.exists(attendance_file):
        attendance_data.to_csv(attendance_file, index=False)
    else:
        attendance_data.to_csv(attendance_file, mode="a", header=False, index=False)

    print(f"Attendance marked for {name} at {timestamp}")


# Load dataset and detect faces with augmentation
faces, labels = load_and_detect_faces(dataset_path, augment=True)

# Train the Random Forest classifier and save the model
rf_classifier, label_encoder = train_rf_classifier(faces, labels)

print("Random Forest Classifier is ready for face recognition.")

# Real-time Face Recognition with webcam (using OpenCV)
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect faces in the frame
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces_detected = cv2.CascadeClassifier(
        cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
    ).detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces_detected:
        face = frame[y : y + h, x : x + w]
        face = cv2.resize(face, (160, 160)) / 255.0  # Normalize the face

        # Extract face embedding using DeepFace
        result = DeepFace.represent(
            frame, model_name="VGG-Face", enforce_detection=False
        )
        face_embedding = result[0]["embedding"]

        # Flatten the embedding
        face_embedding = np.array(face_embedding).reshape(1, -1)

        # Recognize the face using the Random Forest classifier
        prediction = rf_classifier.predict(face_embedding)
        predicted_label = label_encoder.inverse_transform(prediction)

        # Mark attendance
        mark_attendance(predicted_label[0])

        # Draw a rectangle around the face and label it
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(
            frame,
            f"{predicted_label[0]}",
            (x, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.8,
            (0, 255, 0),
            2,
        )

    cv2.imshow("Real-time Face Recognition", frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


Random Forest Classifier is ready for face recognition.
